In [2]:
from pyscf import gto, scf, mcscf

In [3]:
geom = """
O1  0.0000  0.0000  0.0000;
O2  1.0833  0.6667  0.0000;
O3 -1.0833  0.6667  0.0000;
"""

### Checking spin of ground state
Yesterday I didn't check the multiplicity of the ground state, just assumed it was a singlet.

First, will check SCF solutions and then move on to CAS. Including ROHF because this could be convenient for CAS.

In [4]:
spin_states = {"singlet": {"2S": 0, "mf": scf.RHF},
               "triplet": {"2S": 2, "mf": scf.UHF},
               "restricted triplet": {"2S": 2, "mf": scf.ROHF},}

In [5]:
basis_sets = {"TZVP": "def2-tzvp",
              "TZVPD": "def2-tzvpd",}

In [6]:
results = {}

In [7]:
for mult, method in spin_states.items():        
    for basis, basis_set in basis_sets.items():
        name = f"{mult}: {method['mf'].__name__}/{basis}"
        mol = gto.M(atom=geom, basis=basis_set, spin=method["2S"]).build()
        mf = method["mf"](mol).density_fit()
        mf.kernel()
        results[name] = {"mol": mol, "mf": mf}

converged SCF energy = -224.351710462103
converged SCF energy = -224.353165062652
converged SCF energy = -224.342174752846  <S^2> = 2.0900769  2S+1 = 3.059462
converged SCF energy = -224.343509701458  <S^2> = 2.0929718  2S+1 = 3.0613538
converged SCF energy = -224.314493410818
converged SCF energy = -224.315509804624


At the mean field level, it seems like the ground state is a singlet. The ROHF triplet is pretty hopeless but should improve a lot when we start using MCSCF.

Will try CAS(6,6)SCF initially in the interests of time (laptop is old...).

In [25]:
for mult, method in spin_states.items():        
    for basis, basis_set in basis_sets.items():
        mf_name = f"{mult}: {method['mf'].__name__}/{basis}"
        for n_elec in [6]:
            cas_name = f"{mult}: CAS({n_elec}/{n_elec})SCF//{method['mf'].__name__}/{basis}"
            if results.get(cas_name) is None:
                mc = mcscf.DFCASSCF(results[mf_name]["mf"], n_elec, n_elec)
                mc.kernel()
                results[cas_name] = mc

CASSCF energy = -224.525856398727
CASCI E = -224.525856398727  E(CI) = -12.3872682842975  S^2 = 0.0000000
CASSCF energy = -224.526984468453
CASCI E = -224.526984468453  E(CI) = -12.3943743558779  S^2 = 0.0000000


/Users/ch1tk/PROJECTS/ozone/.venv/lib/python3.11/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


CASSCF energy = -224.481702808956
CASCI E = -224.481702808956  E(CI) = -12.4560780069408  S^2 = 2.0000000
CASSCF energy = -224.482752125915
CASCI E = -224.482752125915  E(CI) = -12.4636919551723  S^2 = 2.0000000
CASSCF energy = -224.481702809615
CASCI E = -224.481702809615  E(CI) = -12.4558831778356  S^2 = 2.0000000
CASSCF energy = -224.482752124785
CASCI E = -224.482752124785  E(CI) = -12.4635822991200  S^2 = 2.0000000


In [26]:
for mult, method in spin_states.items():        
    for basis, basis_set in basis_sets.items():
        for n_elec in [6]:
            cas_name = f"{mult}: CAS({n_elec}/{n_elec})SCF//{method['mf'].__name__}/{basis}"
            print(f"{cas_name}: {results[cas_name].e_tot}")


singlet: CAS(6/6)SCF//RHF/TZVP: -224.5258563987266
singlet: CAS(6/6)SCF//RHF/TZVPD: -224.52698446845278
triplet: CAS(6/6)SCF//UHF/TZVP: -224.4817028089556
triplet: CAS(6/6)SCF//UHF/TZVPD: -224.48275212591525
restricted triplet: CAS(6/6)SCF//ROHF/TZVP: -224.4817028096151
restricted triplet: CAS(6/6)SCF//ROHF/TZVPD: -224.48275212478495


* The gap between singlet and triplets has gotten a quite bit bigger at this level of theory. The gap between singlet RHF and triplet UHF was about 0.01 Eh (~0.3 eV), whereas above it is about 0.04 Eh (~1.1 eV).
* The UHF- and ROHF-based calculations seem to have converged to the same solution (energies are the same to ~1e-9 Eh).
* Looking at the docs, this is probably because I've used density fitting to speed up the calculations and the DFCASSCF implementation restricts the $\alpha$ and $\beta$ orbitals to be spatially symmetric, like in ROHF.

As a quick check to see if the restricted nature of the DF-CASSCF implementation is a big problem, I can compare the results of a non-density fitted, (restricted) CASSCF with its (unrestricted) UCASSCF counterpart. The error from density fitting should only be small but for safety, better to compare like with like.

In [31]:
mcscf.CASSCF(results["restricted triplet: ROHF/TZVP"]["mf"], 6, 6).kernel()

CASSCF energy = -224.481702809615
CASCI E = -224.481702809615  E(CI) = -12.4558831778356  S^2 = 2.0000000


(-224.4817028096151,
 -12.455883177835602,
 FCIvector([[-1.54874632e-05,  8.17819051e-06, -9.76975091e-01,
             -3.04721943e-04,  3.68289075e-06, -1.66250437e-05,
              5.90598538e-07, -7.17950968e-02,  1.30196532e-07,
              1.13829628e-06, -4.09801609e-07, -5.17485641e-08,
              3.26386388e-02, -1.04147141e-07,  1.95535876e-02],
            [-6.77296557e-09, -2.31175980e-02,  2.27869622e-07,
              1.47735472e-07, -7.17459982e-03,  1.22185116e-08,
             -1.64481896e-03, -3.11708341e-07,  1.01587694e-06,
              1.60368160e-10, -1.22480445e-09,  8.82101637e-07,
             -2.45460011e-07,  2.85960130e-04, -1.36181715e-08],
            [ 5.09704831e-07, -3.62636002e-07,  4.98882974e-02,
             -6.29067673e-05, -1.86651799e-07,  7.66775026e-07,
              7.26671583e-07, -8.60110088e-02,  1.18899753e-07,
              1.46642236e-06,  1.04318877e-06, -1.41989678e-07,
             -6.76618147e-02,  2.52598647e-07, -2.08396012e

In [8]:
mcscf.UCASSCF(results["triplet: UHF/TZVP"]["mf"], 6, 6).kernel()

UCASSCF energy = -224.4999314646
UCASCI E = -224.4999314646  E(CI) = -12.5229651331208  S^2 = 4.0205321


(-224.4999314646001,
 -12.522965133120806,
 FCIvector([[-1.50742609e-01,  2.26762982e-03, -9.41768011e-01,
              3.35949987e-02, -1.17188403e-03, -8.92914385e-02,
             -1.88344858e-03, -1.37741750e-01, -1.76958885e-03,
              2.38889746e-02,  3.29506867e-02,  3.64617946e-04,
             -1.26875465e-01,  3.51929663e-03, -6.51669101e-03],
            [-3.25792513e-03, -9.05529806e-03,  1.55030998e-03,
             -2.56744222e-04,  1.24381928e-02,  3.45694059e-03,
              4.32131706e-04, -1.88170810e-03, -1.73379341e-02,
             -4.72907107e-04,  5.88731342e-05,  1.57631905e-02,
              2.35849416e-03, -7.14140628e-04, -2.68801727e-05],
            [ 9.19445463e-04,  2.62509471e-02,  1.33128708e-03,
              1.83382610e-04, -2.38135720e-02, -1.96451944e-03,
             -3.88676920e-03, -4.28978008e-03,  5.12245936e-02,
              8.34442538e-04, -4.88182382e-04, -4.18878545e-02,
              2.82215644e-03,  1.50672247e-03, -2.06313014e

* Not sure what to conclude from this because the unrestricted CAS result is extremely spin contaminated (S^2 = ~4 rather than the expected 2)
* Quite a large difference between the restricted and unrestricted CAS results, ~0.02 Eh (~0.5 eV)
* The unrestricted CAS triplet is still quite a lot higher than the singlet
* Overall quite concerning that the relative energies of the states varies between these chosen representations so much. If I want to look at excited state properties, need balanced representations of all states.
*  Perhaps need to investigate some alternate guess orbitals, maybe from DFT or natural orbitals from MP2.
* This could also be an issue with the choice of active space being too restrictive to represent the states adequately, so I could try an automated active space selection method like AVAS or DMET.
* May need to consider using state averaging in the SCF process too